# (Imports)

In [1]:
# https://stackoverflow.com/questions/17077494/how-do-i-convert-a-ipython-notebook-into-a-python-file-via-commandline
from typing import Any, Optional, Union, List, Tuple, Dict, Iterable, Mapping, Callable, Type, Literal
from numbers import Number
from collections.abc import Collection, Sequence
from dataclasses import dataclass, field
from pprint import pprint
from types import FunctionType
import re
import sys
import logging
import functools
from pprint import pformat
import dis
from icecream import ic
#ic.configureOutput(includeContext=True)
map_ = map
reduce_ = functools.reduce

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

# https://stackoverflow.com/a/47024809/1141958
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Combinators-101

This workshop is about building functions with functions.

* function types.
* functions as values.
* information-hiding with functions.
* function that create functions.
* functions that combine functions into new functions.

This workshop presents higher-order functional programming using Python.  While there are other programming which are specifically functional, Python is the most popular programming language in the world and it supports functional programming very well.

# Types

This workshop will use Python's `typing` framework.  Types help us understand what combinators do with functions.

## Basic Types

* `int` - an integer
* `str` - a string
* `Any` - any type
* `List` - a list
* `List[T]` - a `List` containing values of `T`

## Function Types

* `Callable` - anything that can be called with zero or more arguments
* `Callable[..., T]` - a `Callable` with zero or more arguments that returns a value of type `T`
* `Callable[[A1, A2], T]` - Two arguments of type `A1` and `A2`, respectively, returning type `T`

In [2]:
# Example:
# A `Callable` that has 2 parameters,
#   an `str` and `int`
#   and returns a `Tuple` of `int` and `int`:
Callable[[str, int], Tuple[int, int]]

def h(a: str, b: int) -> Tuple[int, int]:
  return (len(a), len(a) * b)

h("ab", 21)

typing.Callable[[str, int], typing.Tuple[int, int]]

(2, 42)

# First-Class Functions

can be:

* assigned to variables
* arguments to functions
* returned as values

## Functions dont have Names, Names have Functions!

In [3]:
print(2)
g = print
g
g(2)

2


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

2


## Where did that function come from?

In [4]:
def call_func_three_times(func: Callable):
  for x in range(3):
    func(x)

call_func_three_times(print)

0
1
2


In [5]:
call_func_three_times(g)

0
1
2


## The most useful useless function

In [6]:
def identity(x: Any) -> Any:
  'Returns the first argument.'
  return x

identity(2)
h = identity
h
h(3)

2

<function __main__.identity(x: Any) -> Any>

3

## Anonymous Functions

In [7]:
lambda x: x + 3
(lambda x: x + 3)(2)           # Never had a name.

<function __main__.<lambda>(x)>

5

In [8]:
plus_three = lambda x: x + 3   # Gave it a name.
plus_three
plus_three(2)

<function __main__.<lambda>(x)>

5

# Closures

* Functions with closure are information hiders.
* They have access to values that are otherwise not visible outside of them.
* They "close over variables".

## Stateless Closures

### Constantly return a value

In [9]:
def constantly(val: Any) -> Callable[[], Any]:
  return lambda : val

In [10]:
constantly_5 = constantly(5)
constantly_5()
constantly_5()

5

5

In [11]:
# Fails with arguments
try:
  constantly_5(13, 17)
except Exception as exc:
  exc

TypeError('constantly.<locals>.<lambda>() takes 0 positional arguments but 2 were given')

### A more robust version

In [12]:
# Functions with zero or more arguments that return anything.
Variadic = Callable[..., Any]

def constantly(x: Any) -> Variadic:
  'Returns a function that returns a constant value.'
  return lambda *_args, **_kwargs: x

constantly_7 = constantly(7)
constantly_7
constantly_7()
constantly_7(13, 17)

<function __main__.constantly.<locals>.<lambda>(*_args, **_kwargs)>

7

7

### Adapters

### Indexable Getters

In [13]:
# Function with one argument that returns anything.
Unary = Callable[[Any], Any]

# A value `x` that supports `x[i]`:
Indexable = Union[List, Tuple, Dict]

In [14]:
def at(i: Any) -> Unary:
  'Returns a function `f(x)` that returns `x[i]`.'
  return lambda x: x[i]

a = [0, 1, 2, 3]
f = at(2)
f(a)

2

In [15]:
def indexed(x: Indexable) -> Unary:
  'Returns a function `f(i)` that returns `x[i]`.'
  return lambda i: x[i]

a = [0, 1, 2, 3]
f = indexed(a)
f(2)

2

#### Works with Strings and Dicts

In [16]:
s = "abcdef"
indexed("abcdef")(4)
at(3)(s)

d = {"a": 2, "b": 3}
indexed(d)("b")
at("b")(d)


'e'

'd'

3

3

### Object Accessors

In [17]:
@dataclass
class Position:
  x: int = 0
  y: int = 0

p = Position(2, 3)
p

def getter(name: str, *default) -> Unary:
  return lambda obj: \
    getattr(obj, name, *default)

def object_get(obj):
  return lambda name, *default: \
    getattr(obj, name, *default)

g = getter('x')
g(p)

h = object_get(p)
h('x')

Position(x=2, y=3)

2

2

In [18]:
h('z', 999)

999

In [19]:
try:
  h('z')
except AttributeError as e:
  print(repr(e))

AttributeError("'Position' object has no attribute 'z'")


In [20]:
def setter(name: str) -> Unary:
  return lambda obj, val: setattr(obj, name, val)

def accessor(name: str) -> Callable:
  def g(obj, *val):
    if val:
      return setattr(obj, name, val[0])
    return getattr(obj, name)
  return g

In [21]:
s = setter('x')
s(p, 5)
p

Position(x=5, y=3)

In [22]:
a = accessor('x')
a(p)
a(p, 7)
p

5

Position(x=7, y=3)

In [23]:
def projection(key: Any, default: Any = None) -> Callable:
  'Returns a function `f(a)` that returns `a.get(key, default)`.'
  return lambda a: a.get(key, default)

## Stateful Functions

### Generators

A generator is an impure function, which may return a different value regardless of its arguments.
Examples:
- random number generator
- reading lines from a file 

In [24]:
def counter(start: int = 0, increment: int = 1) -> Callable[[], int]:
  def g() -> int:
    nonlocal start
    result = start
    start += increment
    return result
  return g

In [25]:
c = counter(2, 3)
c()
c()
c()

2

5

8

# Second-Order Functions


Second-Order Functions return other functions.

They often have the form:

```python
def f(a: Any, ...):
  return lambda b: Any, ...: \
    do_something_with(a, b)
```

or

```python
def f(a: Any, ...):
  def g(b: Any, ...):  # `g` has access to `a`
    return do_something_with(a, b)
  return g
```

## Combinators

Combinators:

* are functions that construct functions from other functions.
* provides a powerful mechanism for reusing logic...
  without having to anticpate the future.

A combinator `c` may have the form:

```python
def c(f: Callable, ...) -> Callable:
  return lambda b, ...: f(do_something_with(a, b))
```

or

```python
def c(f: Callable, ...) -> Callable:
  def g(b, ...):
    return f(do_something_with(a, b))
  return g
```

# Stateful Combinators

In [26]:
def with_counter(f: Callable, i: int = 0) -> Callable:
  'Returns a Callable that applies a counter to f.'
  c = counter(i)
  return lambda *args, **kwargs: \
    f(c(), *args, **kwargs)

def multiply(x, y):
  return x * y

f = with_counter(multiply, 21)
[f(2), f(2), f(3), f(5)]

[42, 44, 69, 120]

# Predicates

In [27]:
# Functions with zero or more arguments that return a boolean.
Predicate = Callable[..., bool]

def is_string(x: Any) -> bool:
  'Returns true if `x` is a string.'
  return isinstance(x, str)

is_string("hello")
is_string(3)

True

False

# Predicate Combinators

In [28]:
# Functions that take a Predicate and return a new Predicate.
PredicateCombinator = Callable[[Predicate], Predicate]

def not_(f: Predicate) -> Predicate:
  'Returns a function that logically negates the result of the given function.'
  return lambda *args, **kwargs: not f(*args, **kwargs)

h = not_(is_string)
h("hello")
h(3)

False

True

# Partial Application

Partial application adds "default" values to a function.

In [29]:
def partial(f: Callable, *args, **kwargs) -> Callable:
  'Returns a function that prepends `args` and merges `kwargs`.'
  def g(*args2, **kwargs2):
    return f(*(args + args2), **dict(kwargs, **kwargs2))
  return g

def add_and_multiply(a, b, c):
  return (a + b) * c

add_and_multiply(2, 3, 5)

h = partial(add_and_multiply, 2)
h(3, 5)

25

25

## Methods are Partially Applied Functions

In [30]:
a = 2
b = 3
a + b
a.__add__(b)    # eqv. to `a + b`
h = a.__add__
h
h(7)

5

5

<method-wrapper '__add__' of int object at 0x1055b3580>

9

In [31]:
def partial_right(f: Callable, *args, **kwargs) -> Callable:
  'Returns a function that appends `args` and merges `kwargs`.'
  def g(*args2, **kwargs2):
    return f(*(args2 + args), **(kwargs2 | kwargs))
  return g

def add_and_multiply(a, b, c):
  return (a + b) * c

add_and_multiply(2, 3, 5)

h = partial_right(add_and_multiply, 2)
h(3, 5)

25

16

In [32]:
def fixed_point(f: Unary) -> Unary:
  'Returns a function that iterates until `f(x) == x`.'
  def g(x: Any) -> Any:
    while True:
      y = x
      x = f(x)
      if x == y:
        return x
  return g

def f(xy):
  ic(xy)
  x, y = xy
  return (x.replace(y, y[1:]), y[1:])

g = fixed_point(f)
g(("abccabaaxabc", "abc"))



ic| xy: ('abccabaaxabc', 'abc')
ic| xy: ('bccabaaxbc', 'bc')
ic| xy: ('ccabaaxc', 'c')
ic| xy: ('abaax', '')


('abaax', '')

In [33]:
def f(e, x):
  ic(x)
  return 0.5 * (x + e / x)

ic(x := 50)
g = fixed_point(partial(f, x))
ic(y := g(x * 2))
ic(y * y)

ic| x := 50: 50


50

ic| x: 100
ic| x: 50.25
ic| x: 25.622512437810947
ic| x: 13.786960692097871
ic| x: 8.70678790224796
ic| x: 7.22471690979458
ic| x: 7.072701650643464
ic| x: 7.071068000579016
ic| x: 7.071067811865477
ic| x: 7.0710678118654755
ic| y := g(x * 2): 7.0710678118654755


7.0710678118654755

ic| y * y: 50.00000000000001


50.00000000000001

# Manipulating Sequences

## Mapping functions over sequences

In [34]:
# Note: Python has a built-in `map` -- this is for illustration:
def map(f: Unary, xs: Sequence) -> Sequence:
  'Returns a sequence of `f(x)` for each element `x` in `xs`.'
  acc = []
  for x in xs:
    acc.append(f(x))
  return acc

items = [1, "string", False, True, None]
items
map(identity, items)
map(constantly_7, items)
map(is_string, items)
map(not_(is_string), items)
map(plus_three, [3, 5, 7, 11])

[1, 'string', False, True, None]

[1, 'string', False, True, None]

[7, 7, 7, 7, 7]

[False, True, False, False, False]

[True, False, True, True, True]

[6, 8, 10, 14]

In [35]:
# Restore the built-in:
# map = map_

## Filtering Sequences with Predicates

In [36]:
def filter(f: Unary, xs: Sequence) -> Sequence:
  'Returns a sequence of the elements of `xs` for which `f` returns true.'
  return [x for x in xs if f(x)]

items = [1, "string", False, True, None]
filter(is_string, items)
filter(not_(is_string), items)

['string']

[1, False, True, None]

## Reducing Sequences with Binary Functions

In [37]:
# Functions with two arguments that return anything.
Binary = Callable[[Any, Any], Any]

def reduce(f: Binary, init: Any, xs: Sequence) -> Sequence:
  'Returns the result of `init = f(x, init)` for each element `x` in `xs`.'
  for x in xs:
    init = f(init, x)
  return init

def add(x, y):
  return x + y

reduce(add, 2, [3, 5, 7])
a_list_of_strings = ["A", "List", 'Of', 'Strings']
reduce(add, "Here Is ", a_list_of_strings)

17

'Here Is AListOfStrings'

In [38]:
items = [1, "string", 2, 3, "-and-more", 5]

# Concat all strings:
reduce(add, "", filter(is_string, items))

# Sum of all numbers:
def is_number(x: Any) -> bool:
  return not isinstance(x, bool) and isinstance(x, Number)
reduce(add, 0, filter(is_number, items))

# Sum all non-strings:
reduce(add, 0, filter(not_(is_string), items))

'string-and-more'

11

11

In [39]:
def conjoin(a, b) -> Callable[[Any, Any], Tuple[Any, Any]]:
  'Creates a Tuple from two arguments.'
  return (a, b)

items = [3, "a", 5, "b", 7, "c", 11, True]
reduce(conjoin, 2, items)

dict(map(with_counter(conjoin, 21), ["a", "b", "c", "d"]))

((((((((2, 3), 'a'), 5), 'b'), 7), 'c'), 11), True)

{21: 'a', 22: 'b', 23: 'c', 24: 'd'}

## Map as a Reduction

In [40]:
def map_r(f: Unary, xs: Sequence) -> Sequence:
  def acc(seq, x):
    return seq + [f(x)]
  return reduce(acc, [], xs)

map(plus_three, [3, 5, 7, 11])
map_r(plus_three, [3, 5, 7, 11])

[6, 8, 10, 14]

[6, 8, 10, 14]

## Filter as a Reduction

In [41]:
def filter_r(f: Unary, xs: Sequence) -> Sequence:
  def acc(seq, x):
    return seq + [x] if f(x) else seq
  return reduce(acc, [], xs)

items
filter(is_string, items)
filter_r(is_string, items)

[3, 'a', 5, 'b', 7, 'c', 11, True]

['a', 'b', 'c']

['a', 'b', 'c']

## Mapcat (aka Flat-Map)

In [42]:
ConcatableUnary = Callable[[Any], Sequence]

def mapcat(f: ConcatableUnary, xs: Sequence):
  'Concatenate the results of `map(f, xs)`.'
  return reduce(add, [], map(f, xs))

def duplicate(n, x):
  return [x] * n

duplicate_each_3_times = partial(mapcat, partial(duplicate, 3))
duplicate_each_3_times([".", "*"])
duplicate_each_3_times(range(4, 7))

['.', '.', '.', '*', '*', '*']

[4, 4, 4, 5, 5, 5, 6, 6, 6]

## Manipulating Arguments

In [43]:
def reverse_args(f: Callable) -> Callable:
  def g(*args, **kwargs):
    return f(*reversed(args), **kwargs)
  return g

def divide(x, y):
  return x / y

divide(2, 3)
reverse_args(divide)(2, 3)

reduce(reverse_args(add), " reversed ", a_list_of_strings)
reduce(reverse_args(conjoin), 2, [3, 5, 7])

0.6666666666666666

1.5

'StringsOfListA reversed '

(7, (5, (3, 2)))

# Function Composition

In [44]:
def compose(*callables) -> Variadic:
  'Returns the composition one or more functions, in reverse order.'
  'For example, `compose(g, f)(x, y)` is equivalent to `g(f(x, y))`.'
  f: Callable = callables[-1]
  gs: Sequence[Unary] = tuple(reversed(callables[:-1]))
  def h(*args, **kwargs):
    result = f(*args, **kwargs)
    for g in gs:
      result = g(result)
    return result
  return h

def multiply_by_3(x):
  return x * 3

plus_three(multiply_by_3(5))

h = compose(plus_three, multiply_by_3)
h(5)

18

18

In [45]:
def juxt(*fs):
    "Returns a function that applies each f in fs to its arguments."
    def g(*args, **kwargs):
        return map(lambda f: f(*args, **kwargs), fs)
    return g

def negative(x):
    return - x

def repeat(n, x):
    return [x] * n

map(juxt(identity, negative, partial(repeat, 3)), [2, 3, 5, 7])

[[2, -2, [2, 2, 2]],
 [3, -3, [3, 3, 3]],
 [5, -5, [5, 5, 5]],
 [7, -7, [7, 7, 7]]]

# Interlude

In [46]:
def modulo(modulus: int) -> Callable[[int], int]:
  return lambda x: x % modulus

mod_3 = modulo(3)
map(mod_3, range(10))

[0, 1, 2, 0, 1, 2, 0, 1, 2, 0]

In [47]:
h = compose(indexed(a_list_of_strings), mod_3)
map(h, range(10))

['A', 'List', 'Of', 'A', 'List', 'Of', 'A', 'List', 'Of', 'A']

## Arity Reduction

In [48]:
def unary(f: Variadic) -> Unary:
  return lambda *args, **kwargs: f((args, kwargs))

h = unary(identity)
h()
h(1)
h(1, 2)
h(a=1, b=2)

((), {})

((1,), {})

((1, 2), {})

((), {'a': 1, 'b': 2})

# Developer Affordance

## Debugging

In [49]:
def tracer(
    name: str = "?",
    log: Unary | None = None,
  ) -> Callable:
  indent = 0
  logger = log or (lambda msg: sys.stderr.write(f'  ## {msg}\n'))
  def g(f, *args, **kwargs):
    nonlocal indent
    indent += 1
    msg = f"{' ' * indent}{name}({format_args(args, kwargs)})"
    logger(f"{msg} => ...")
    result = f(*args, **kwargs)
    logger(f"{msg} => {result!r}")
    indent -= 1
    return result
  return g

def trace(f: Callable, name: str = "", *args) -> Callable:
  if not name:
    name = f.__name__
  t = tracer(name, *args)
  return lambda *args, **kwargs: t(f, *args, **kwargs)

def format_args(args, kwargs):
  return ', '.join(list(map(repr, args)) + [f'{k}={v!r}' for k, v in kwargs])

h = compose(str, plus_three)
map(h, [2, 3, 5])

g = trace(h, "g")
map(g, [2, 3, 5])

map_g = trace(partial(map, g), "map_g")
map_g([2, 3, 5])


['5', '6', '8']

  ##  g(2) => ...
  ##  g(2) => '5'
  ##  g(3) => ...
  ##  g(3) => '6'
  ##  g(5) => ...
  ##  g(5) => '8'


['5', '6', '8']

  ##  map_g([2, 3, 5]) => ...
  ##  g(2) => ...
  ##  g(2) => '5'
  ##  g(3) => ...
  ##  g(3) => '6'
  ##  g(5) => ...
  ##  g(5) => '8'
  ##  map_g([2, 3, 5]) => ['5', '6', '8']


['5', '6', '8']

## Error Handlers

In [50]:
def except_(f: Variadic, ex_class, error: Unary) -> Callable:
  def g(*args, **kwargs):
    try:
      return f(*args, **kwargs)
    except ex_class as exc:
      return error((exc, args, kwargs))
  return g

h = except_(plus_three, TypeError, compose(partial(logging.error, 'plus_three: %s'), repr))
h(2)
h('Nope')

5

ERROR:root:plus_three: (TypeError('can only concatenate str (not "int") to str'), ('Nope',), {})


# Web Application Architecture

Application middleware combinators inspired Python WSGI and Ruby Rack.

- An "App" is anything callable with a single dict argument.
- It receives a "Request"
  - typically a Dict of input: headers, body and customary values passed along an "application stack".
- It returns an HTTP "Response": Tuple of:
  - numeric HTTP status code
  - dict of HTTP headers
  - body -- a sequence of response body chunks
- applications and middleware follow the same protocol.
- Combinators create new Apps by wrapping others.


In [51]:
Status = int
Headers = Dict[str, Any]
Body = Iterable
Req = Dict[str, Any]
Res = Tuple[Status, Headers, Body]
App = Callable[[Req], Res]

## Simple Applications

### Hello, World!

In [52]:
def hello_world_app(req: Req) -> Res:
  return 200, {}, ("Hello, World!",)
app = hello_world_app
app({})

(200, {}, ('Hello, World!',))

### Do Something Useful

In [53]:
def something_useful_app(req: Req) -> Res:
  x, y = req['input.data']
  return 200, {}, (x * y,)

app = something_useful_app
app({'input.data': [2, 5]})


(200, {}, (10,))

In [54]:
app = something_useful_app
app({'input.data': ["ab", 3]})

(200, {}, ('ababab',))

## Application Combinators

Input combinators follow this pattern:

In [55]:
def compose_input_handler(app: App) -> App:
  def input_handler(req: Req) -> Res:
    # do something with req...
    return app(req)
  return input_handler

Output combinators follow this pattern:

In [56]:
def compose_output_handler(app: App) -> App:
  def output_handler(req: Req) -> Res:
    status, headers, body = response = app(req)  # <<<
    # do something with response...
    return status, headers, body
  return output_handler

## Tracing

In [57]:
def trace_app(app: App, ident="", stream=sys.stderr) -> App:
    "Traces requests and responses."
    def indent(msg):
        stream.write(f"{'  ' * TRACE_INDENT[0]}{msg}")
    def log(msg):
        indent(f" #{msg} {ident}\n")
    def pp(data):
        indent("")
        pprint(data, stream=stream)
    def tracing(req):
        log(">>>")
        TRACE_INDENT[0] += 1
        pp(req)
        result = app(req)
        log("...")
        pp(result)
        TRACE_INDENT[0] -= 1
        log("<<<")
        return result
    return tracing
TRACE_INDENT = [0]

In [58]:
app = something_useful_app
app = trace_app(app, 'my_app')
app({'input.data': [5, 7]})

 #>>> my_app
  {'input.data': [5, 7]}
   #... my_app
  (200, {}, (35,))
 #<<< my_app


(200, {}, (35,))

## Exception Handling

In [59]:
def capture_exception(app: App, cls=Exception, status=500) -> App:
    def capturing_exception(req: Req) -> Res:
        try:
            return app(req)
        except cls as exc:
            return status, {"Content-Type": "text/plain"}, (repr(exc),)
    return capturing_exception

In [60]:
app = something_useful_app
app = capture_exception(app)
app({'input.data': [{"a": 1}, 7]})

(500,
 {'Content-Type': 'text/plain'},
 ('TypeError("unsupported operand type(s) for *: \'dict\' and \'int\'")',))

## Reading Inputs, Writing Outputs

In [61]:
Content = str
Data = Any

def read_input(app: App, read: Callable[[Data], Content]) -> App:
    "Reads body.stream"
    def reader(req: Req) -> Res:
        req["input.content"] = read(req["input.stream"])
        return app(req)
    return reader

## Decoding Inputs, Encoding Outputs

In [62]:
Encoder = Callable[[Data], Content]
Decoder = Callable[[Content], Data]

def decode_content(app: App, decoder: Decoder, content_types=None, strict=False) -> App:
    """
    Decodes body with decoder(input.content) for content_types.
    If strict and Content-Type is not expected, return 400.
    """

    def decoding_content(req: Req) -> Res:
        req["input.data"] = decoder(req["input.content"])
        content_type = req.get("Content-Type")
        if strict and content_types and content_type not in content_types:
            msg = f"Unexpected Content-Type {content_type!r} : expected: {content_types!r} : "
            return 400, {"Content-Type": 'text/plain'}, (msg,)
        return app(req)
    return decoding_content


def encode_content(app: App, encoder: Encoder, content_type="text/plain") -> App:
    "Encodes body with encoder.  Sets Content-Type."
    def encoding_content(req: Req) -> Res:
        status, headers, body = app(req)
        content = "".join(map(encoder, body))
        headers |= {
            "Content-Type": content_type,
            "Content-Length": len(content),
        }
        return status, headers, [content]
    return encoding_content


## Decode JSON, Encode JSON

In [63]:
import json

def decode_json(app: App, **kwargs) -> App:
    "Decodes JSON content."
    def decoding_json(content: Content) -> Any:
        return json.loads(content, **kwargs)
    return decode_content(app, decoding_json, content_types={'application/json', 'text/plain'}, strict=True)


def encode_json(app: App, **kwargs) -> App:
    "Encodes data as JSON."
    def encoding_json(data: Data) -> Content:
        return json.dumps(data, **kwargs) + "\n"
    return encode_content(app, encoding_json, content_type='application/json')

## Simple App Handles JSON!

In [64]:
app = something_useful_app
# app = trace_app(app, 'hello_world_app')
app = decode_json(app)
# app = trace_app(app, 'decode_json')
app = encode_json(app)
# app = trace_app(app, 'encode_json')
app({'input.content': "[11, 13]", "Content-Type": 'application/json'})

(200, {'Content-Type': 'application/json', 'Content-Length': 4}, ['143\n'])

# Logical Combinators

## Predicators

In [65]:
def re_pred(pat: str, re_func: Callable = re.search) -> Predicate:
  'Returns a predicate that matches a regular expression.'
  rx = re.compile(pat)
  return lambda x: re_func(rx, str(x)) is not None

re_pred("ab")("abc")
re_pred("ab")("nope")

True

False

In [66]:
def default(f: Variadic, g: Variadic) -> Variadic:
  def h(*args, **kwargs):
    if (result := f(*args, **kwargs)) is not None:
      return result
    return g(*args, **kwargs)
  return h

asdf

## Logical Predicate Composers

In [67]:
def and_(f: Variadic, g: Variadic) -> Variadic:
  'Returns a function `h(x, ...)` that returns `f(x, ...) and g(x, ...).'
  return lambda *args, **kwargs: f(*args, **kwargs) and g(*args, **kwargs)

def or_(f: Variadic, g: Variadic) -> Variadic:
  'Returns a function `h(x, ...)` that returns `f(x, ...) or g(x, ...).'
  return lambda *args, **kwargs: f(*args, **kwargs) or g(*args, **kwargs)

def is_int(x):
  return isinstance(x, int)

def is_string(x):
  return isinstance(x, str)

is_word = re_pred(r'^[a-z]+$')

# If x is an int, add three:
h = and_(is_int, plus_three)
# If x is a string, is it a word?:
g = and_(and_(is_string, is_word), len)
# One or the other:
func = or_(h, g)
items = ["hello", "not-a-word", 2, 3.5, None]
map(juxt(identity, func), items)


[['hello', 5], ['not-a-word', False], [2, 5], [3.5, False], [None, False]]

In [68]:
Procedure = Callable[[], Any]

def if_(f: Variadic, g: Unary, h: Unary) -> Variadic:
  def i(*args, **kwargs):
    if (result := f(*args, **kwargs)):
      return g()
    return h()
  return i

# Interpreters

## Operator Predicates

In [69]:

def binary_op(op: str) -> Optional[Callable[[Any, Any], Any]]:
  'Returns a function for a binary operator by name.'
  return BINARY_OPS.get(op)

BINARY_OPS = {
  '+': lambda a, b: a + b,
  '-': lambda a, b: a - b,
  '*': lambda a, b: a * b,
  '/': lambda a, b: a / b,
  '==': lambda a, b: a == b,
  '!=': lambda a, b: a != b,
  '<':  lambda a, b: a < b,
  '>':  lambda a, b: a > b,
  '<=': lambda a, b: a <= b,
  '>=': lambda a, b: a >= b,
  'and': lambda a, b: a and b,
  'or': lambda a, b: a or b,
}
for n, f in BINARY_OPS.items():
  f.__name__ = f"binary_op({n!r})"

binary_op('==') (2, 2)
binary_op('!=') (2, 2)

True

False

In [70]:
# Create a table where `x OP y` is true:
[
  f'{x} {op} {y}'
  for op in ['<', '==', '>']
  for x in (2, 3, 5)
  for y in (2, 3, 5)
  if binary_op(op)(x, y)
]

['2 < 3',
 '2 < 5',
 '3 < 5',
 '2 == 2',
 '3 == 3',
 '5 == 5',
 '3 > 2',
 '5 > 2',
 '5 > 3']

In [ ]:
def stacky(program):
    'A stack-oriented expression evaluator.'
    def eval(stack, item):
        if isinstance(item, int):
            return stack + [item]
        else:
            return stack[:-2] + [item(stack[-2], stack[-1])]
    def parse(word):
        if re.match(r'-?\d+', word):
            return int(word)
        return binary_op(word)
    return reduce(trace(eval, "eval"), [], map(trace(parse, "parse"), program.split(" ")))

stacky("33 2 3 + 5 * >")


  ##  parse('33') => ...
  ##  parse('33') => 33
  ##  parse('2') => ...
  ##  parse('2') => 2
  ##  parse('3') => ...
  ##  parse('3') => 3
  ##  parse('+') => ...
  ##  parse('+') => <function <lambda> at 0x1210bf1a0>
  ##  parse('5') => ...
  ##  parse('5') => 5
  ##  parse('*') => ...
  ##  parse('*') => <function <lambda> at 0x1210bf2e0>
  ##  parse('>') => ...
  ##  parse('>') => <function <lambda> at 0x1210bf600>
  ##  eval([], 33) => ...
  ##  eval([], 33) => [33]
  ##  eval([33], 2) => ...
  ##  eval([33], 2) => [33, 2]
  ##  eval([33, 2], 3) => ...
  ##  eval([33, 2], 3) => [33, 2, 3]
  ##  eval([33, 2, 3], <function <lambda> at 0x1210bf1a0>) => ...
  ##  eval([33, 2, 3], <function <lambda> at 0x1210bf1a0>) => [33, 5]
  ##  eval([33, 5], 5) => ...
  ##  eval([33, 5], 5) => [33, 5, 5]
  ##  eval([33, 5, 5], <function <lambda> at 0x1210bf2e0>) => ...
  ##  eval([33, 5, 5], <function <lambda> at 0x1210bf2e0>) => [33, 25]
  ##  eval([33, 25], <function <lambda> at 0x1210bf600>) =

[True]

In [72]:
def op_pred(op: str, b: Any) -> Predicate | None:
  'Returns a predicate function given an operator name and a constant.'
  if pred := binary_op(op):
    return lambda a: pred(a, b)
  if op == "not":
    return lambda a: not a
  if op == "~=":
    return re_pred(b)
  if op == "~!":
    return not_(re_pred(b))
  return None

In [73]:
h = op_pred(">", 3)
h(2)
h(5)

False

True

In [74]:
g = op_pred("~=", 'ab+c')
g('')
g('ab')
g('abbbcc')

False

False

True

## Sequencing

In [75]:
def progn(*fs: Sequence[Callable]) -> Callable:
  'Returns a function that calls each function in turn and returns the last result.'
  def g(*args, **kwargs):
    result = None
    for f in fs:
      result = f(*args, **kwargs)
    return result
  return g

In [76]:
def prog1(f0: Callable, *fs: Sequence[Callable]) -> Callable:
  'Returns a function that calls each function in turn and returns the last result.'
  def g(*args, **kwargs):
    result = f0(*args, **kwargs)
    for f in fs:
      result = f(*args, **kwargs)
    return result
  return g

In [77]:
def reverse_apply(x: Any) -> Callable:
  return lambda f, *args, **kwargs: f(x, *args, **kwargs)

reverse_apply(1) (plus_three)

4

## Parser Combinators

In [78]:
# Parser input: a sequence of lexemes:
Input = Sequence[Any]

# A parsed value and remaining input:
Parsed = Tuple[Any, Input]

# A parser matches the input sequence and produces a result or nothing:
Parser = Callable[[Input], Parsed | None]

In [79]:

def show_match(p: Parser) -> Variadic:
  def g(input: Input):
    return (p(input) or False, '<=', input)
  return g

In [80]:
first = at(0)
def rest(x: Input) -> Input:
  return x[1:]

def equals(x) -> Parser:
  'Returns a parser that matches `x`.'
  def g(input: input):
    y = first(input)
    logging.debug("equals(%s, %s)", repr(x), repr(y))
    if x == y:
      return y, rest(input)
  return g

h = equals('a')
h(['a'])
h(['b', 2])


DEBUG:root:equals('a', 'a')


('a', [])

DEBUG:root:equals('a', 'b')


In [81]:
def which(p: Predicate) -> Parser:
  'Returns a parser for the next lexeme when `p(lexeme)` is true.'
  def g(input: Input):
    if p(first(input)):
      return (first(input), rest(input))
  return g

g = which(is_string)
g(['a'])
g([2])
g(['a', 'b'])

def alternation(*ps) -> Parser:
  def g(input):
    for p in ps:
      if (result := p(input)):
        return result
  return g

g = alternation(which(is_string), which(is_number))
g(['a'])
g([2])
g([False])


('a', [])

('a', ['b'])

('a', [])

(2, [])

## Sequence Parsers

In [82]:
ParsedSequence = Tuple[Sequence, Input]
SequenceParser = Callable[[Input], ParsedSequence | None]

def one(p: Parser) -> SequenceParser:
  'Returns a parser for one lexeme.'
  def g(input: Input):
    if input and (result := p(input)):
      parsed, input = result
      return [parsed], input
  return g

g = one(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])


(['a'], [])

(['a'], ['b'])

In [83]:
def zero_or_more(p: Parser) -> SequenceParser:
  'Returns a parser for zero or more lexemes.'
  def g(input: Input):
    acc = []
    while input and (result := p(input)):
      parsed, input = result
      acc.append(parsed)
    return acc, input
  return g

g = zero_or_more(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

([], [])

(['a'], [])

([], [2])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [84]:
def one_or_more(p: Parser) -> SequenceParser:
  'Returns a parser for one or more lexemes as a sequence.'
  p = zero_or_more(p)
  def g(input: Input):
    if (result := p(input)) and len(result[0]) >= 1:
      return result
  return g

In [85]:
g = one_or_more(which(is_string))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

(['a'], [])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [86]:
def sequence_of(*parsers) -> SequenceParser:
  'Returns a parser for parsers of a sequence.'
  def g(input: Input):
    acc = []
    for p in parsers:
      if result := p(input):
        parsed, input = result
        acc.extend(parsed)
      else:
        return None
    return acc, input
  return g

g = sequence_of(one(which(is_string)), one(which(is_string)))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 'b', 2])
g(['a', 'b', 3, 5])

(['a', 'b'], [])

(['a', 'b'], [2])

(['a', 'b'], [3, 5])

In [87]:
g = sequence_of(one_or_more(which(is_number)))
g([])
g(['a'])
g([2])
g([2, 3])
g([2, 3, False])

([2], [])

([2, 3], [])

([2, 3], [False])

In [88]:
g = sequence_of(one(which(is_string)), one_or_more(which(is_number)))
g([])
g(['a'])
g([2])
g(['a', 'b'])
g(['a', 2])
g(['a', 2, 3])
g(['a', 2, 'b', 3])
g(['a', 2, 3, False])
g(['a', 2, 3, False, 'more'])

(['a', 2], [])

(['a', 2, 3], [])

(['a', 2], ['b', 3])

(['a', 2, 3], [False])

(['a', 2, 3], [False, 'more'])

## Parser Grammar

In [89]:
def take_while(f: Unary) -> Unary:
  def g(seq):
    acc = []
    while seq and (x := f(seq[0])):
      acc.append(x)
    return acc
  return g

def action(p: Parser, f: Callable[[Any], Any]) -> Parser:
  'Returns a parser that calls `f` with parsed value.'
  def g(seq):
    if result := p(seq):
      return a(result[0]), result[1]
  return g

env = {}
def a(v):
  # nonlocal x
  env['x'] = v
  return v
f = action(which(is_string), a)
f(["abc"])
env

('abc', [])

{'x': 'abc'}

## Lexical Scanning

In [122]:
def eat(rx: str):
  p = re.compile(rx)
  return lambda s: re.sub(p, '', s)

def lexeme(pat: str, post = at(0)):
  post = post or (lambda m: m[0])
  rx = re.compile(pat)
  ws = eat(r'^\s+')
  def g(input):
    input = ws(input)
    if input and (m := re.match(rx, input)):
      return post(m), input[len(m[0]):]
  return g

In [125]:
def grammar_parser():
  env = {}
  def _(id):
    return lambda *args: env[id](*args)

  def action(p: Parser, action: Unary) -> Parser:
    def g(input: Input):
      if result := p(input):
        value = action(result[0])
        return value, result[1]
    return g

  def cache(p: Parser) -> Parser:
    '''
    Cache the result of p(input).
    This improves performance when a definition is
    parsed multiple times.
    '''
    d = {}
    def g(input):
      if v := d.get(id(input)):
        # logging.debug('cached : %s => %s', repr(input), repr(v[0]))
        return v[0]
      v = p(input)
      d[id(input)] = (v,)
      return v
    return g

  def definition(id, p, act=None):
    # p = trace(p, id)
    # p = cache(p)
    a = None
    if act is True:
      a = lambda x: (id, x)
    elif isinstance(act, str):
      a = lambda x: (act, x)
    elif act:
      a = act
    if a:
      p = action(p, a)
    p = cache(p)
    env[id] = p

  definition('string',        lexeme(r'^"(\\"|\\?[^"]+)*"',
                                lambda m: ('string', eval(m[0]))))
  definition('regex',         lexeme(r'^/((\\/|(\\?[^/]+))*)/',
                                lambda m: ('regex', re.compile(m[1]))))
  definition('name',          lexeme(r'^[a-zA-Z][a-zA-Z0-9_]*'))
  definition('terminal',      alternation(_("string"), _('regex')))
  definition('non_terminal',  _('name'),
                                lambda x: ('reference', x))
  definition('action',        lexeme(r'^@\{(.+?)\}@',
                                lambda m: ('action', m[1].strip())))
  definition('basic_match',   alternation(_('non_terminal'), _('terminal')))
  definition('bound_match',   sequence_of(one(_('basic_match')), one(lexeme(r'^:')), one(_('name'))),
                                lambda x: ('bound_match', x[0], x[2]))
  definition('match',         alternation(_('bound_match'), _('basic_match')))
  definition('matches',       one_or_more(_('match')))
  definition('matches_with_action', sequence_of(_('matches'), _('action')))
  definition('pattern',       alternation(_('matches_with_action'), _('matches')))
  definition('sequence',      sequence_of(_('pattern')), lambda x: ('sequence', x))
  definition('alternation',   sequence_of(one(_('sequence')), one(lexeme(r'\|')), one(_('production'))),
                                lambda x: ('alternation', x[0], x[2]))
  definition('production',    alternation(_('alternation'), _('sequence')))
  definition('definition',    sequence_of(one(_('name')), one(lexeme(r"=")), one(_('production')), one(lexeme(r';'))),
                                lambda x: ('definition',  x[0], x[2]))
  definition('grammar',       sequence_of(one_or_more(_('definition'))), 'grammar')

  def g(input, start=None):
    return _(start or 'grammar')(input)
  return g

grammar_tests = [
  ['"asdf"', 'pattern'],
  ['  a = b c;'],
  ['a =b c|d;'],
  ['a = b c | d | e f;'],
  ['a = "foo";'],
  ['b = /^foo/:c;'],
  # ['a = b : x c : y @{ do_this(x, y) }@ ;']
]

gram = grammar_parser()

for grammar_test in grammar_tests:
  logging.debug("============================================\n")
  ic(grammar_test)
  result = gram(*grammar_test)
  ic(result)


DEBUG:root:============================================

ic| grammar_test: ['"asdf"', 'pattern']


['"asdf"', 'pattern']

ic| result: ([('string', 'asdf')], '')


([('string', 'asdf')], '')

DEBUG:root:============================================

ic| grammar_test: ['  a = b c;']


...

(('grammar', [('definition', 'a', ('sequence', [('string', 'foo')]))]), '')

DEBUG:root:============================================

ic| grammar_test: ['b = /^foo/:c;']


['b = /^foo/:c;']

ic| result: (('grammar',
              [('definition',
                'b',
                ('sequence', [('bound_match', ('regex', re.compile('^foo')), 'c')]))]),
             '')


(('grammar',
  [('definition',
    'b',
    ('sequence',
     [('bound_match', ('regex', re.compile(r'^foo', re.UNICODE)), 'c')]))]),
 '')

## Grammar Compiler

In [133]:
def compile_grammar(gram):
  gensym_i = 0
  def gensym(name):
    nonlocal gensym_i
    gensym_i += 1
    return f"__{name}__{gensym_i}"

  input = gensym('input')
  result = gensym('result')
  newlines = "\n\n"
  depth = 0

  stack = []
  def push():
    stack.append((input, result))

  def pop():
    input, result = stack.pop()

  def indent(s, chars):
    prefix = f"\n{' ' * chars}"
    return prefix.join(map(lambda s: f"{prefix}{x}", s.split("\n")))

  def advance(value, remaining = f"{result}[1]"):
    return f"""
    {result} = ({value}, {remaining})
    {input} = {result}[1]
  """

  def grammar(definitions):
    ic(definitions)
    return f"""
def parse({input}):
{newlines.join(map(compile, definitions))}
"""

  def definition(name, production):
    return f"""
  def {name}({input}):
    {result} = None
{compile(production)}
    return {result}
"""

  def alternation(*exprs):
    code = ""
    input_save = gensym('input')
    code += f"""
    {input_save} = {input}
"""
    for expr in exprs:
      code += compile(expr)
      push()
      result = gensym('result')
      code += f"""
{compile(expr)}
    if {result}:
      return {result}
    {input} = {input_save}
"""
      pop()
    code += f"""
      return {result}
"""
    return code

  def sequence(exprs):
    sequence = gensym('sequence')
    code = ""
    code += f"""
    {sequence} = []
"""
    for expr in exprs:
      code += f"""
{compile(expr)}
    if not {result}:
      return None
    {sequence}.append({result}[0])
{advance(sequence)}
"""
    return code

  def reference(name):
    return f"""
    {result} = {name}({input})
"""

  def patterns(exprs):
    return "\n".join(map(compile, exprs))

  def bound_match(expr, name):
    return f"""
{compile(expr)}
    {name} = {result} and {result}[0]
"""

  def matches_with_action(expr):
    ic(expr)
    return f"""
{compile(expr)}
    if {result}:
      return {indent(action, 6)}, {input}
"""

  def string(s):
    const = repr(s)
    check = f"({input}[0:{len(s)}] == {const})"
    return f"""
    if not {check}:
      return None
{advance(const, f"{input}[:{len(s)}]")}
"""

  def integer(expr):
    match = gensym('match')
    return f"""
    if not ({match} := re.search(r'^([-+]?\d+)', {input})):
      return None
{advance(f"int({match}[1])", f"{input}[:len({match}[1])]")}
"""

  def regex(expr):
    match = gensym('match')
    return f"""
    if not ({match} := {expr!r}.search({input}):
      return None
{advance(f"{match}[1]", f"{input}[0:len({match}[1])]")}
"""

  def action(*exprs):
    ic(("action", exprs))
    return """FAIL"""

  funcs = locals()
  def compile(expr):
    nonlocal depth
    depth += 1
    indent = '  ' * depth
    ic(("compile", expr))
    result = funcs[expr[0]](*expr[1:])
    depth -= 1
    return f"""
{indent}####################################
{indent}# {repr(expr)} (((
{result}
{indent}# {repr(expr)} )))
{indent}####################################
"""

  return compile(gram)

gram = grammar_parser()
for grammar_test in grammar_tests:
  ic(grammar_test)
  grammar_parsed = gram(*grammar_test)[0]
  ic(grammar_parsed)
  print(compile_grammar(grammar_parsed))


ic| grammar_test: ['"asdf"', 'pattern']


['"asdf"', 'pattern']

ic| grammar_parsed: [('string', 'asdf')]


[('string', 'asdf')]

ic| ("compile", expr): ('compile', [('string', 'asdf')])


KeyError: ('string', 'asdf')

In [ ]:
# Grammar V2
# WORK-IN-PROGRESS
def grammar_parser():
  gensym_i = 0
  def gensym(name):
    nonlocal gensym_i
    gensym_i += 1
    return f"__{name}__{gensym_i}"

  definitions = {}
  def _(id):
    return lambda *args: definitions[id](*args)

  def cache(p: Parser) -> Parser:
    '''
    Cache the result of p(input).
    This improves performance when a definition is
    parsed multiple times.
    '''
    d = {}
    def g(input):
      if v := d.get(input):
        # logging.debug('cached : %s => %s', repr(input), repr(v[0]))
        return v[0]
      v = p(input)
      d[input] = (v,)
      return v
    return g

  def definition(id, p, act=None):
    log = partial(logging.debug, '  # %s')
    # p = trace(id, log, p)
    # p = cache(p)
    a = None
    if act is True:
      a = lambda x: (id, x)
    elif isinstance(act, str):
      a = lambda x: (act, x)
    elif act:
      a = act
    if a:
      p = action(p, a)
    p = cache(p)
    definitions[id] = p

  def match_string(m):
    s = m[1]
    def g(input):
      if input.starts_with(s):
        return s, input[:len(s)]
    return g

  def match_regex(m):
    rx = re.compile(m[1])
    def g(input):
      if m := re.search(rx, input):
        return s, input[:len(m[0])]
    return g

  def reference(id):
    return lambda input: definitions[id](input)

  env = {}

  def action(m):
    body = m[1].strip()
    func_name = gensym('action')
    expr = ''
    expr += f"""
  def {func_name}(env):
"""
    for name, value in env:
      expr += f"""
    {name} = env[{name!r}]
"""
    expr += indent(body, 4)
    return expr

  def bound_match(x):
    parser, _, name = x
    def g(input):
      value = env[name] = parser(input)
      return value
    return g

  def set_env(x):
    k, _, v = x
    env[k] = v
    return v


  definition('string',        lexeme(r'^"(\\"|\\?[^"]+)*"', match_string))
  definition('regex',         lexeme(r'^/((\\/|(\\?[^/]+))*)/', match_regex))
  definition('name',          lexeme(r'^[a-zA-Z][a-zA-Z0-9_]*'))
  definition('terminal',      alternation(_("string"), _('regex')))
  definition('non_terminal',  _('name'), reference)
  definition('action',        lexeme(r'^@\{(.+?)\}@', action))
  definition('basic_match',   alternation(_('non_terminal'), _('terminal')))
  definition('bound_match',   sequence_of(one(_('basic_match')), one(lexeme(r'^:')), one(_('name'))), bound_match)
  definition('match',         alternation(_('bound_match'), _('basic_match')))
  definition('matches',       one_or_more(_('match')))
  definition('matches_with_action', sequence_of(_('matches'), _('action')))
  definition('pattern',       alternation(_('matches_with_action'), _('matches')))
  definition('sequence',      sequence_of(_('pattern')), lambda x: ('sequence', x))
  definition('alternation',   sequence_of(one(_('sequence')), one(lexeme(r'\|')), one(_('production'))),
                                lambda x: ('alternation', x[0], x[2]))
  definition('production',    alternation(_('alternation'), _('sequence')))
  definition('definition',    sequence_of(one(_('name')), one(lexeme(r"=")), one(_('production')), one(lexeme(r';'))),
                                set_env)
  definition('grammar',       sequence_of(one_or_more(_('definition'))), 'grammar')

  def g(input, start=None):
    return _(start or 'grammar')(input)
  return g

gram = grammar_parser()

TypeError: grammar_parser.<locals>.action() takes 1 positional argument but 2 were given

In [ ]:
def test(input, start=None):
  logging.debug("============================================\n")
  ic((start, input))
  result = gram(input, start)
  ic((start, input))
  ic(result)
  parser = compile_grammar(result[0])
  logging.debug(parser)

test('"asdf"', 'pattern')
test('  a = b c;')
test('a =b c|d;')
# test('a = b c | d | e f;')
# test('a = "foo";')
# test('b = /foo/:c;')
# test('a = b : x c : y @{ do_this(x, y) }@ ;')